Something that's made me think he's how much sample_wise_preprocessing helped with improving the learning rate. 
My thoughts are now on how much feature wise preprocessing might help  

In [1]:
import os
import sys


up1 = os.path.abspath('../../utils/') 
up2 = os.path.abspath('../../models/') 
sys.path.insert(0, up1)
sys.path.insert(0, up2)

In [2]:
from ucf101_data_utils import get_test_data_opt_flow, get_train_data_opt_flow
from motion_network import getKerasCifarMotionModel2, getKerasCifarMotionModelOnly
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from keras.optimizers import SGD
import cv2 
import numpy as np
import keras

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [20]:
from optical_flow_data_gen import DataGenerator
from ucf101_data_utils import get_test_data_opt_flow, get_train_data_opt_flow

training_options_1 = { 'rescale' : 1./255, 
                     'samplewise_center': False, 
                         'samplewise_std_normalization':False,
                            'zoom_range' : 0.2, 
                            'horizontal_flip' : True}

validation_options_1 = { 'rescale' : 1./255,
                     'samplewise_center': False, 
                         'samplewise_std_normalization':False}




params_train_1 = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size': 64,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : False,
                'enable_augmentation' : True,
               'training_opts' : training_options_1}

params_valid_1 = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size':64,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : True,
               'validation_opts' : validation_options_1}

id_labels_train = get_train_data_opt_flow('../../data/ucf101_splits/trainlist01_small.txt')
labels = id_labels_train[1]
id_test = get_test_data_opt_flow('../../data/ucf101_splits/testlist01_small.txt', \
                       '../../data/ucf101_splits/classInd_small.txt')

training_gen_no_standardization = DataGenerator(*id_labels_train, **params_train_1)


validation_gen_no_standardization = DataGenerator(id_test[0], id_test[1], **params_valid_1)

In [21]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.initializers import Ones
from keras import optimizers

def getModel(lr=1e-2):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 2)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    model.add(Activation('sigmoid'))

    optimizers.SGD(lr=lr)
    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])
    return model

In [22]:
model_no_std = getModel(lr=1e-2)

In [23]:
mod1 = model_no_std.fit_generator(generator=training_gen_no_standardization, steps_per_epoch=32,
                    validation_data=validation_gen_no_standardization, validation_steps=32,
                    use_multiprocessing=True,
                    workers=4, epochs=5,
                    verbose=1)

Epoch 1/5
32/32 [==============================] - 22s 702ms/step - loss: 2.3039 - acc: 0.0967 - val_loss: 2.3003 - val_acc: 0.1206
Epoch 2/5
32/32 [==============================] - 19s 590ms/step - loss: 2.3022 - acc: 0.0933 - val_loss: 2.2999 - val_acc: 0.1201
Epoch 3/5
32/32 [==============================] - 19s 588ms/step - loss: 2.2999 - acc: 0.1201 - val_loss: 2.2985 - val_acc: 0.1230
Epoch 4/5
32/32 [==============================] - 19s 591ms/step - loss: 2.2992 - acc: 0.1143 - val_loss: 2.2976 - val_acc: 0.1216
Epoch 5/5
32/32 [==============================] - 19s 608ms/step - loss: 2.2990 - acc: 0.1108 - val_loss: 2.2971 - val_acc: 0.1221


In [31]:


training_options_2 = { 'rescale' : 1./255, 
                     'samplewise_center': True, 
                         'samplewise_std_normalization':True,
                            'zoom_range' : 0.2, 
                            'horizontal_flip' : True}

validation_options_2 = { 'rescale' : 1./255,
                     'samplewise_center': True, 
                         'samplewise_std_normalization':True}




params_train_2 = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size': 128,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : False,
                'enable_augmentation' : True,
               'training_opts' : training_options_2}

params_valid_2 = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size':128,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : True,
               'validation_opts' : validation_options_2}

id_labels_train = get_train_data_opt_flow('../../data/ucf101_splits/trainlist01_small.txt')
labels = id_labels_train[1]
id_test = get_test_data_opt_flow('../../data/ucf101_splits/testlist01_small.txt', \
                       '../../data/ucf101_splits/classInd_small.txt')

training_gen_with_standardization = DataGenerator(*id_labels_train, **params_train_2)


validation_gen_with_standardization = DataGenerator(id_test[0], id_test[1], **params_valid_2)

In [32]:
model_with_std = getModel(lr=1e-2)

In [30]:
mod2 = model_with_std.fit_generator(generator=training_gen_with_standardization, steps_per_epoch=32,
                    validation_data=validation_gen_with_standardization, validation_steps=32,
                    use_multiprocessing=True,
                    workers=4, epochs=5,
                    verbose=1)

Epoch 1/5
32/32 [==============================] - 25s 775ms/step - loss: 2.2819 - acc: 0.1279 - val_loss: 2.2654 - val_acc: 0.1436
Epoch 2/5
32/32 [==============================] - 21s 670ms/step - loss: 2.2550 - acc: 0.1489 - val_loss: 2.2433 - val_acc: 0.1675
Epoch 3/5
32/32 [==============================] - 21s 657ms/step - loss: 2.2179 - acc: 0.1865 - val_loss: 2.2151 - val_acc: 0.2368
Epoch 4/5
32/32 [==============================] - 21s 653ms/step - loss: 2.2004 - acc: 0.2310 - val_loss: 2.1978 - val_acc: 0.2217
Epoch 5/5
32/32 [==============================] - 21s 654ms/step - loss: 2.1748 - acc: 0.2192 - val_loss: 2.1636 - val_acc: 0.2671


In [33]:
mod2_ = model_with_std.fit_generator(generator=training_gen_with_standardization, steps_per_epoch=32,
                    validation_data=validation_gen_with_standardization, validation_steps=32,
                    use_multiprocessing=True,
                    workers=4, epochs=5,
                    verbose=1)

Epoch 1/5
32/32 [==============================] - 51s 2s/step - loss: 2.2606 - acc: 0.1130 - val_loss: 2.2481 - val_acc: 0.1450
Epoch 2/5
32/32 [==============================] - 50s 2s/step - loss: 2.2188 - acc: 0.1501 - val_loss: 2.2220 - val_acc: 0.1533
Epoch 3/5
32/32 [==============================] - 45s 1s/step - loss: 2.1899 - acc: 0.1819 - val_loss: 2.2161 - val_acc: 0.2434
Epoch 4/5
32/32 [==============================] - 49s 2s/step - loss: 2.1768 - acc: 0.2273 - val_loss: 2.1788 - val_acc: 0.2756
Epoch 5/5
32/32 [==============================] - 45s 1s/step - loss: 2.1424 - acc: 0.2493 - val_loss: 2.1577 - val_acc: 0.2886


So its obvious standardization offers a huge benefit. My feeling though is that with a small batch sample you are essentially getting goodish scaling, but you are not improving the search within the feature space globally.

## Data generator with Feature scaling 

In [15]:


training_options_3 = { 'rescale' : 1./255,
                            'zoom_range' : 0.2, 
                            'horizontal_flip' : True }

validation_options_3 = { 'rescale' : 1./255 }




params_train_3 = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size': 64,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : False,
                'enable_augmentation' : False,
                  'feature_wise_standardization' : True,
               'training_opts' : training_options_3}

params_valid_3 = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size':64,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                  'feature_wise_standardization' : True,
                'n_classes' : 10,
           'validation' : True,
               'validation_opts' : validation_options_3}

id_labels_train = get_train_data_opt_flow('../../data/ucf101_splits/trainlist01_small.txt')
labels = id_labels_train[1]
id_test = get_test_data_opt_flow('../../data/ucf101_splits/testlist01_small.txt', \
                       '../../data/ucf101_splits/classInd_small.txt')

training_gen_with_feature_std = DataGenerator(*id_labels_train, **params_train_3)


validation_gen_with_feature_std = DataGenerator(id_test[0], id_test[1], **params_valid_3)

In [16]:
training_gen_with_feature_std.fit()

In [17]:
validation_gen_with_feature_std.fit()

In [18]:
model_with_fw_std = getModel(lr=1e-2)

In [19]:
mod3 = model_with_fw_std.fit_generator(generator=training_gen_with_feature_std, steps_per_epoch=32,
                    validation_data=validation_gen_with_feature_std, validation_steps=32,
                    use_multiprocessing=True,
                    workers=4, epochs=10,
                    verbose=1)

Epoch 1/10
32/32 [==============================] - 16s 513ms/step - loss: 2.3483 - acc: 0.1006 - val_loss: 2.3038 - val_acc: 0.0913
Epoch 2/10
32/32 [==============================] - 15s 463ms/step - loss: 2.3034 - acc: 0.1099 - val_loss: 2.3000 - val_acc: 0.1226
Epoch 3/10
32/32 [==============================] - 15s 467ms/step - loss: 2.3021 - acc: 0.1035 - val_loss: 2.2986 - val_acc: 0.1230
Epoch 4/10
32/32 [==============================] - 14s 449ms/step - loss: 2.3021 - acc: 0.1040 - val_loss: 2.2993 - val_acc: 0.1255
Epoch 5/10
32/32 [==============================] - 15s 456ms/step - loss: 2.3022 - acc: 0.1079 - val_loss: 2.2986 - val_acc: 0.1064
Epoch 6/10
32/32 [==============================] - 14s 450ms/step - loss: 2.2999 - acc: 0.1104 - val_loss: 2.2983 - val_acc: 0.1245
Epoch 7/10
32/32 [==============================] - 14s 443ms/step - loss: 2.2969 - acc: 0.1182 - val_loss: 2.2953 - val_acc: 0.1226
Epoch 8/10
32/32 [==============================] - 16s 487ms/step - 

In [10]:



training_options_4 = { 'rescale' : 1./255, 
                     'samplewise_center': True, 
                         'samplewise_std_normalization':True,
                            'zoom_range' : 0.2, 
                            'horizontal_flip' : True}

validation_options_4 = { 'rescale' : 1./255,
                     'samplewise_center': True, 
                         'samplewise_std_normalization':True}



params_train_4 = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size': 64,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                'n_classes' : 10,
           'validation' : False,
                'enable_augmentation' : True,
                  'feature_wise_standardization' : True,
               'training_opts' : training_options_4}

params_valid_4 = { 'data_dir' : "/data/tvl1_flow",
          'dim': (224,224),
          'batch_size':64,
          'n_frames': 1,
          'n_frequency': 1,
          'shuffle': True, 
                  'feature_wise_standardization' : True,
                'n_classes' : 10,
           'validation' : True,
               'validation_opts' : validation_options_4}

id_labels_train = get_train_data_opt_flow('../../data/ucf101_splits/trainlist01_small.txt')
labels = id_labels_train[1]
id_test = get_test_data_opt_flow('../../data/ucf101_splits/testlist01_small.txt', \
                       '../../data/ucf101_splits/classInd_small.txt')

training_gen_with_fw_std_sample = DataGenerator(*id_labels_train, **params_train_4)


validation_gen_with_fw_std_sample = DataGenerator(id_test[0], id_test[1], **params_valid_4)

In [11]:
training_gen_with_fw_std_sample.fit()

In [12]:
validation_gen_with_fw_std_sample.fit()

In [13]:
model_with_fw_std_sample = getModel(lr=1e-2)

In [14]:
mod4 = model_with_fw_std_sample.fit_generator(generator=validation_gen_with_fw_std_sample, steps_per_epoch=32,
                    validation_data=training_gen_with_fw_std_sample, validation_steps=32,
                    use_multiprocessing=True,
                    workers=4, epochs=10,
                    verbose=1)

Epoch 1/10
32/32 [==============================] - 26s 798ms/step - loss: 2.3198 - acc: 0.1011 - val_loss: 2.3046 - val_acc: 0.1113
Epoch 2/10
32/32 [==============================] - 25s 792ms/step - loss: 2.3032 - acc: 0.1143 - val_loss: 2.3024 - val_acc: 0.1138
Epoch 3/10
32/32 [==============================] - 27s 839ms/step - loss: 2.3023 - acc: 0.1235 - val_loss: 2.3023 - val_acc: 0.1133
Epoch 4/10
32/32 [==============================] - 25s 797ms/step - loss: 2.3021 - acc: 0.1260 - val_loss: 2.3022 - val_acc: 0.1138
Epoch 5/10
32/32 [==============================] - 26s 807ms/step - loss: 2.3020 - acc: 0.1206 - val_loss: 2.3021 - val_acc: 0.1138
Epoch 6/10
32/32 [==============================] - 27s 838ms/step - loss: 2.3018 - acc: 0.1255 - val_loss: 2.3019 - val_acc: 0.1167
Epoch 7/10
32/32 [==============================] - 25s 781ms/step - loss: 2.3017 - acc: 0.1245 - val_loss: 2.3019 - val_acc: 0.1123
Epoch 8/10
32/32 [==============================] - 25s 789ms/step - 